## Convert old IMAS at NERSC to new IMAS 
June 21, 2024 

Venkitesh Ayyar

In [23]:
import numpy as np
import datetime
import os

import pymongo
import pprint
from bson.objectid import ObjectId

import yaml
import json

In [2]:
# ## Convert json to yaml for easy viewing
# with open('gyrokinetics_new.json', 'r') as j:
#     GK_dict = json.loads(j.read())
    
# with open('gyrokinetics_new.yaml','w') as f:
#     yaml.dump(GK_dict,f,default_flow_style=False)

In [3]:
import xmltodict

## Load both yaml files

In [4]:
fname='gyro_imas_old.yaml'
with open(fname,'r') as f:
    gk_dict = yaml.safe_load(f)
    
fname='gyro_imas_new.yaml'
with open(fname,'r') as f:
    GK_dict = yaml.safe_load(f)

In [5]:
print(gk_dict.keys(),'\n',GK_dict.keys())

dict_keys(['code', 'collisions', 'flux_surface', 'fluxes_integrated_norm', 'ids_properties', 'model', 'species', 'species_all', 'wavevector']) 
 dict_keys(['code', 'collisions', 'flux_surface', 'ids_properties', 'linear', 'max_repr_length', 'model', 'non_linear', 'normalizing_quantities', 'species', 'species_all', 'time', 'version'])


## Print dictionary hierarchy

In [6]:
def f_print_dict_struct(dict1,tab_count=0):
    if isinstance(dict1,dict):
        for i,j in dict1.items():
            if not isinstance(j,dict):
                print('\t'*tab_count, i,'\t',type(j))
                if (isinstance(j,list) and len(j)>0):
                    if isinstance(j[0],dict):
                        print('\t'*tab_count,i,'[0]:')                    
                        f_print_dict_struct(j[0],tab_count+1)
            else: 
                # print('\t'*tab_count,"------------")
                print('\t'*tab_count,i,':')
                f_print_dict_struct(j, tab_count+1)

                
# f_print_dict_struct(gk_dict,0)

# f_print_dict_struct(GK_dict,0)
    

In [ ]:
for key in GK_dict.keys():
    print(key)

In [ ]:
gk_dict['code']['parameters']

In [ ]:
GK_dict['code']['parameters']

In [ ]:
xmltodict.parse(GK_dict['code']['parameters'])

In [ ]:
keys='code'
for key in gk_dict[keys]:
    print(key)
print('\n\n')
for key in GK_dict[keys]:
    print(key)

In [ ]:
GK_dict[keys].keys(),gk_dict[keys].keys()

In [ ]:
gk_dict['species'][0].keys()

In [ ]:
GK_dict['species'][0].keys()

In [ ]:
gk_dict.keys()


In [ ]:
# ## modification for species 
# key1='species'

# common_keys   = [i for i in gk_dict[key1][0].keys() if i in GK_dict[key1][0].keys()]
# old_only_keys = [i for i in gk_dict[key1][0].keys() if i not in GK_dict[key1][0].keys()]
# new_only_keys = [i for i in GK_dict[key1][0].keys() if i not in gk_dict[key1][0].keys()]

# print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)


In [ ]:
key1='ids_properties'

common_keys   = [i for i in gk_dict[key1].keys() if i in GK_dict[key1].keys()]
old_only_keys = [i for i in gk_dict[key1].keys() if i not in GK_dict[key1].keys()]
new_only_keys = [i for i in GK_dict[key1].keys() if i not in gk_dict[key1].keys()]

print(common_keys,'\n\n',old_only_keys,'\n\n',new_only_keys)

# for i in common_keys: 
#     print(i,'\t',gk_dict[key1][i],'\t',GK_dict[key1][i],'\n')

# for i in old_only_keys:
#     print(i, gk_dict[key1][i])
    
# for i in new_only_keys:
#     print(i, GK_dict[key1][i])

In [ ]:
def f_collisions(old_gk,new_gk):
    ''' conversion for key: collisions '''
    
    dict1={}
    for key in ['max_repr_length','version']: 
        dict1[key] = new_gk[key]
    
    dict1['collisionality_norm']=old_gk['collisionality_norm']
    
    return dict1

def f_flux_surface(old_gk, new_gk):
    ''' conversion for key: flux_surface '''
    
    dict1={}
    
    ## Old keys not present 
    keys = ['triangularity_lower', 'triangularity_upper']
    
    ## New keys not present in old 
    keys = ['delongation_dr_minor_norm', 'dgeometric_axis_r_dr_minor', 'dgeometric_axis_z_dr_minor', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['b_field_tor_sign', 'dc_dr_minor_norm', 'ds_dr_minor_norm', 'elongation', 'ip_sign', 'magnetic_shear_r_minor', 'pressure_gradient_norm', 'q', 'r_minor_norm', 'shape_coefficients_c', 'shape_coefficients_s']
    
    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species_all(old_gk, new_gk):
    ''' conversion for key: species_all '''
    
    dict1={}
    
    ## Old keys not present 
    keys =  ['debye_length_reference', 'zeff'] 
    
    ## New keys not present in old 
    keys = ['angle_pol', 'debye_length_norm', 'max_repr_length', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['beta_reference', 'shearing_rate_norm', 'velocity_tor_norm'] 

    for key in common_keys:
        dict1[key] = old_gk[key]
        
    return dict1

def f_species(old_gk_lst, new_gk_lst):
    ''' conversion for key: species '''
    
    lst=[]
    
    
    for old_d,new_d in zip(old_gk_lst,new_gk_lst):
        dict1={}
        ## Old keys not present 
        keys =  ['name'] 

        ## New keys not present in old 
        keys = ['max_repr_length', 'potential_energy_gradient_norm', 'potential_energy_norm', 'version']
        for key in keys:
            dict1[key] = new_d[key]

        ## Common keys 
        common_keys = ['charge_norm', 'density_log_gradient_norm', 'density_norm', 'mass_norm', 'temperature_log_gradient_norm', 'temperature_norm', 'velocity_tor_gradient_norm'] 

        for key in common_keys:
            dict1[key] = old_d[key]
            
        lst.append(dict1)
            
    return lst


def f_ids_properties(old_gk, new_gk):
    ''' conversion for key: ids_properties '''
    dict1={}
    
    ## Old keys not present 
    keys = ['creator', 'date'] 
    
    ## New keys not present in old 
    keys = ['creation_date', 'homogeneous_time', 'max_repr_length', 'name', 'occurrence_type', 'provenance', 'provider', 'version']
    for key in keys:
        dict1[key] = new_gk[key]
    
    ## Common keys 
    common_keys = ['comment'] 

    for key in common_keys[:]:
        dict1[key] = old_gk[key]
        
    ## Special fixes
    dict1['creation_data']=old_gk['date']
    dict1['provider'] = 'manual'
    
    ## Note: 'creator' not used. Should it be stored in 'name' ? 
    # dict1['name'] = old_gk['creator']

    return dict1


# f_species(gk_dict['species'],GK_dict['species'])
f_ids_properties(gk_dict['ids_properties'],GK_dict['ids_properties'])


In [ ]:
GK_dict['non_linear']

In [7]:
gk_dict.keys()

dict_keys(['code', 'collisions', 'flux_surface', 'fluxes_integrated_norm', 'ids_properties', 'model', 'species', 'species_all', 'wavevector'])

In [21]:
gk_dict['wavevector'][0].keys()
gk_dict['wavevector'][0]['eigenmode'][0].keys()

# gk_dict['wavevector'][0]['radial_component_norm']
# eigenmode is very large 

dict_keys(['a_field_parallel_perturbed_norm_imaginary', 'a_field_parallel_perturbed_norm_real', 'fluxes_norm', 'frequency_norm', 'growth_rate_norm', 'growth_rate_tolerance', 'moments_norm_rotating_frame', 'phi_potential_perturbed_norm_imaginary', 'phi_potential_perturbed_norm_real', 'poloidal_angle'])

In [ ]:
GK_dict['linear']['wavevector'][0].keys()

In [22]:
GK_dict['linear']['wavevector'][0]['version'

dict_keys(['max_repr_length', 'version', 'wavevector'])

In [17]:
GK_dict['linear']['wavevector'][0]['eigenmode'][0].keys()

dict_keys(['angle_pol', 'code', 'fields', 'frequency_norm', 'growth_rate_norm', 'growth_rate_tolerance', 'initial_value_run', 'linear_weights', 'linear_weights_rotating_frame', 'max_repr_length', 'moments_norm_gyrocenter', 'moments_norm_gyrocenter_bessel_0', 'moments_norm_gyrocenter_bessel_1', 'moments_norm_particle', 'poloidal_turns', 'time_norm', 'version'])

## to do : 
- wavevector
- code

In [31]:
par1 = gk_dict['code']['parameters']
par1.keys()

dict_keys(['Bref', 'ENDIANNESS', 'ExBrate', 'GIT_BRANCH', 'GIT_MASTER', 'Lref', 'OMP_NUM_THREADS', 'Omega0_tor', 'PRECISION', 'RELEASE', 'Tref', 'adapt_lx', 'beta', 'calc_dt', 'charge1', 'charge2', 'charge3', 'coll', 'coll_cons_model', 'coll_order', 'collision_op', 'comp_type', 'debye2', 'dens1', 'dens2', 'dens3', 'diag_Blev(0)', 'diag_trap_levels', 'diagdir', 'dpdx_pm', 'dpdx_term', 'dt_max', 'dt_vlasov', 'ev_coll', 'ev_coll_est', 'geomfile', 'hyp_z', 'hypz_compensation', 'init_cond', 'init_time', 'istep_energy', 'istep_field', 'istep_mom', 'istep_nrg', 'istep_omega', 'istep_schpt', 'istep_vsp', 'kx_center', 'ky0_ind', 'kymin', 'lv', 'lw', 'lx', 'ly', 'magn_geometry', 'major_R', 'mass1', 'mass2', 'mass3', 'minor_r', 'mref', 'n0_global', 'n_fields', 'n_moms', 'n_parallel_sims', 'n_pol', 'n_procs_s', 'n_procs_sim', 'n_procs_v', 'n_procs_w', 'n_procs_x', 'n_procs_y', 'n_procs_z', 'n_spec', 'name1', 'name2', 'name3', 'nblocks', 'nky0', 'nltdt_off', 'nonlinear', 'norm_flux_projection', 'nr

In [33]:
# GK_dict['ids_properties']
par2 = xmltodict.parse(GK_dict['code']['parameters'])

In [ ]:
a

In [36]:
par2['root'].keys()

dict_keys(['parallelization', 'box', 'in_out', 'general', 'geometry', 'species', 'info', 'units', 'bsgrid'])

In [37]:
f_print_dict_struct(par2,0)

 root :
	 parallelization :
		 n_procs_s 	 <class 'str'>
		 n_procs_v 	 <class 'str'>
		 n_procs_w 	 <class 'str'>
		 n_procs_x 	 <class 'str'>
		 n_procs_y 	 <class 'str'>
		 n_procs_z 	 <class 'str'>
		 n_procs_sim 	 <class 'str'>
		 n_parallel_sims 	 <class 'str'>
	 box :
		 n_spec 	 <class 'str'>
		 nx0 	 <class 'str'>
		 nky0 	 <class 'str'>
		 nz0 	 <class 'str'>
		 nv0 	 <class 'str'>
		 nw0 	 <class 'str'>
		 kymin 	 <class 'str'>
		 lv 	 <class 'str'>
		 lw 	 <class 'str'>
		 adapt_lx 	 <class 'str'>
		 x0 	 <class 'str'>
		 n0_global 	 <class 'str'>
		 ky0_ind 	 <class 'str'>
	 in_out :
		 diagdir 	 <class 'str'>
		 read_checkpoint 	 <class 'str'>
		 write_checkpoint 	 <class 'str'>
		 istep_field 	 <class 'str'>
		 istep_mom 	 <class 'str'>
		 istep_nrg 	 <class 'str'>
		 istep_omega 	 <class 'str'>
		 istep_vsp 	 <class 'str'>
		 istep_schpt 	 <class 'str'>
		 istep_energy 	 <class 'str'>
		 write_std 	 <class 'str'>
		 iterdb_file 	 <class 'str'>
	 general :
		 nonlinear 	